# Generate Documentation using Amazon Q for Business and evaluate it against existing documentation.

In this notebook we will use Amazon Q for Business to generate a documentation for repo files, ingest them to Amazon Q, get documentation repo, iterate through its files, ask questions to Q about documented functionality and then compare AI-geenrated vs human generated doc.

In [1]:
%pip install boto3 --upgrade 
%pip install GitPython shutils python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 48.3 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.67
    Uninstalling botocore-1.34.67:
      Successfully uninstalled botocore-1.34.67
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.67
    Uninstalling boto3-1.34.67:
      Successfully uninstalled boto3-1.34.67
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.67 requires botocore==1.34.67, but you have botocore 1.34.72 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Using cached GitPython-3.1.42-py3-none-any.whl.metadata (12 kB)
  Preparing metadata (setup.py) ... done
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Us

In [2]:
import os
import boto3
import time
import json
import datetime
import requests
import re
import logging
import shutil
import uuid
import git
from dotenv import load_dotenv

In [3]:
%%writefile .env
TOKEN_NAME=amazon-q-use-case-1-private
DOC_REPO_URL=https://github.com/bayer-int/DSE-DevDocs.git
CODE_REPO_URL=https://github.com/bayer-int/dse-apis.git
DOC_REPO_SUBDIR=docs/platform_developer/APIs
FILE_DOC_SUFFIX=API
USERNAME=sviola-bayer

Writing .env


In [4]:
load_dotenv(override=True)

True

In [9]:
# GitHub authentication token
token_name = os.environ['TOKEN_NAME']

# Repository owner, name etc
# this will be env vars in Lambda
# owner = os.environ['OWNER']
code_repo = os.environ['CODE_REPO_URL']
doc_repo = os.environ['DOC_REPO_URL']
doc_repo_subdir = os.environ['DOC_REPO_SUBDIR']
suffix = os.environ['FILE_DOC_SUFFIX']
username = os.environ['USERNAME']

In [10]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [11]:
secretsmanager = boto3.client('secretsmanager')
amazon_q = boto3.client('qbusiness')

In [12]:
amazon_q_user_id = "splwis@amazon.pl"
amazon_q_app_id = "aac5cf2c-5ecc-4bea-bf74-25caf9636ace"
index_id = "0476a6b0-af9c-418f-9a7c-ffe6b6299379"
role_arn = "arn:aws:iam::643306558745:role/QBusiness-Application-Code-Analysis"

In [38]:
filename = ".ipnybcheckpoint"
if filename.endswith('.md') and not filename.startswith('.'):
    print("yes")

In [13]:
def include_code_file_type(filename):
    if not filename.endswith(('.png', '.jpg', '.jpeg', '.gif', '.zip', '.md')) and not filename.startswith('.'):
        return True
    else:
        return False


def include_doc_file_type(filename):
    if filename.endswith('.md') and not filename.startswith('.'):
        return True
    else:
        return False


def check_if_missing_response(answer):
    if answer == "Sorry, I could not find relevant information to complete your request.":
        return True
    else:
        return False

In [14]:
def clone_repo(repo, token_name, local_path):
    token = get_access_token(token_name)
    _, repo_url = repo.split("https://")
    token_url = f"https://{username}:{token}@{repo_url}"
    git.Repo.clone_from(token_url, local_path)

In [15]:
def ask_question_to_gen_code_doc(prompt, filename):
    data = open(filename, 'rb')
    answer = amazon_q.chat_sync(
        applicationId=amazon_q_app_id,
        userId=amazon_q_user_id,
        userMessage=prompt,
        attachments=[
            {
                'data': data.read(),
                'name': filename
            },
        ],
    )
    return answer['systemMessage']

In [16]:
def ask_question_to_gen_doc_summary(prompt):
    answer = amazon_q.chat_sync(
        applicationId=amazon_q_app_id,
        userId=amazon_q_user_id,
        userMessage=prompt,
    )
    return answer['systemMessage']

In [17]:
def ask_question_to_compare_docs(prompt, human_gen_doc, ai_gen_doc):
    answer = amazon_q.chat_sync(
        applicationId=amazon_q_app_id,
        userId=amazon_q_user_id,
        userMessage=prompt,
        attachments=[
            {
                'data': human_gen_doc,
                'name': "Human-Generated-Summary"
            },
            {
                'data': ai_gen_doc,
                'name': "AI-Generated-Summary"
            },
        ],
    )
    return answer['systemMessage']

In [18]:
def upload_prompt_answer_and_file_name(filename, prompt, answer, repo_url):
    cleaned_file_name = os.path.join(repo_url[:-4], '/'.join(filename.split('/')[2:]))
    amazon_q.batch_put_document(
        applicationId=amazon_q_app_id,
        indexId=index_id,
        roleArn=role_arn,
        documents=[
            {
                'id': str(uuid.uuid5(uuid.NAMESPACE_URL, f"{cleaned_file_name}")),
                'contentType': 'PLAIN_TEXT',
                'title': cleaned_file_name,
                'content': {
                    'blob': f"{cleaned_file_name} | {prompt} | {answer}".encode('utf-8')
                },
                'attributes': [
                    {
                        'name': 'url',
                        'value': {
                            'stringValue': cleaned_file_name
                        }
                    }
                ]
            },
        ]
    )

In [19]:
def get_access_token(secret_name):
    response = secretsmanager.get_secret_value(SecretId=secret_name)
    return response['SecretString']

In [20]:
def save_answers(answer, filepath, folder):
    filepath = f"{filepath}.out"
    with open(f"{folder}/{filepath}", "w") as f:
        f.write(answer)

In [21]:
file_doc_gen_prompts = [
    "Come up with a list of questions and answers about the attached file. Keep answers dense with information. A good question for a database related file would be 'What is the database technology and architecture?' or for a file that executes SQL commands 'What are the SQL commands and what do they do?' or for a file that contains a list of API endpoints 'What are the API endpoints and what do they do?'",

    "Generate comprehensive documentation about the attached file. Make sure you include what dependencies and other files are being referenced as well as function names, class names, and what they do.",

    "Identify anti-patterns in the attached file. Make sure to include examples of how to fix them. Try Q&A like 'What are some anti-patterns in the file?' or 'What could be causing high latency?'",

    "Suggest improvements to the attached file. Try Q&A like 'What are some ways to improve the file?' or 'Where can the file be optimized?'",

    "Please provide description of the attached file. Summarize the intent, resources and capabilities in separate sections.",

    "Please describe each API method. Then list its inputs and outputs in a table. Include sample invocation.",
]

In [22]:
file_doc_summary_gen_prompt = "Please provide me description <functionality> API in <repo_name>. Please include intent within a tag <intent></intent>, resources within a tag <resources></resources>, capabilities within a tag <capabilities></capabilities> in the subsequent paragraphs. In capabilities section you should explain all API methods, what is their purpose, inputs and outputs. Please include as much details as you can. This is important."

In [23]:
file_doc_eval_prompt = "<instruction>Evaluate the AI-Generated-Summary using the Human-Generated-Summary using a 10-point scale. Justify your score. Write your score response in score brackets: <score></score> Then write explanation in explanation brackets: <explanation></explanation> </instruction>"

In [24]:
def should_ignore_path(path):
    path_components = path.split(os.sep)
    for component in path_components:
        if component.startswith('.'):
            return True
        elif component == 'node_modules':
            return True
        elif component == '__pycache__':
            return True
    return False

In [25]:
def parse_repo_url(repo_url):
  
    # Match the pattern github.com/owner/repo
    match = re.search(r'github.com/([^/]+)/([^/]+)', repo_url)
  
    if match:
        owner = match.group(1)
        repo_name = match.group(2)[:-4]
        return owner, repo_name
    else:
        raise ValueError(f"Invalid repo URL: {repo_url}")

In [26]:
def process_code_repo_files(repo_url, prompts):
    # Temporary clone location
    repo_owner, repo_name = parse_repo_url(repo_url)
    tmp_dir = f"tmp/code/{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
    destination_folder = 'repositories/code'

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Clone the repository
    logger.info(f"Cloning repository... {repo_url}")
    clone_repo(repo_url, token_name, tmp_dir)
    logger.info(f"Finished cloning repository {repo_url}")

    # Copy all files to destination folder
    for src_dir, dirs, files in os.walk(tmp_dir):
        dst_dir = src_dir.replace(tmp_dir, destination_folder)
        if not os.path.exists(dst_dir):
            os.mkdir(dst_dir)
        for file_ in files:
            src_file = os.path.join(src_dir, file_)
            dst_file = os.path.join(dst_dir, file_)
            if os.path.exists(dst_file):
                os.remove(dst_file)
            shutil.copy(src_file, dst_dir)

    # Delete temp clone
    shutil.rmtree(tmp_dir)
    processed_files = []
    failed_files = []

    logger.info(f"Processing code files in {destination_folder}")
    for root, dirs, files in os.walk(destination_folder):
        if not should_ignore_path(root):
            for file in files:
                if include_code_file_type(file):

                    file_path = os.path.join(root, file)

                    for attempt in range(3):
                        try:
                            logger.info(f"\033[92mProcessing code file: {file_path}\033[0m")
                            for prompt in prompts:
                                answer = ask_question_to_gen_code_doc(prompt, file_path)
                                upload_prompt_answer_and_file_name(file_path, prompt, answer, repo_url) 
                            # Upload the file itself to the index
                            code = open(file_path, 'r')
                            upload_prompt_answer_and_file_name(file_path, "", code.read(), repo_url)
                            processed_files.append(file)
                            break
                        except Exception as e:
                            logger.error(f"Error: {e}")
                            time.sleep(15)
                    else:
                        logger.info(f"\033[93mSkipping file: {file_path}\033[0m")
                        failed_files.append(file_path)
    return repo_name
    logger.info(f"Processed files: {processed_files}")
    logger.info(f"Failed files: {failed_files}")

In [32]:
def process_doc_repo_files(repo_url, doc_subdir, suffix_exc, gen_doc_summary_prompt, doc_eval_prompt, code_repo):
    # Temporary clone location
    repo_owner, repo_name = parse_repo_url(repo_url)
    tmp_dir = f"tmp/doc/{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
    destination_folder = "repositories/doc"
    eval_folder = "repositories/eval_results"

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Clone the repository
    logger.info(f"Cloning repository... {repo_url}")
    clone_repo(repo_url, token_name, tmp_dir)
    logger.info(f"Finished cloning repository {repo_url}")

    # Copy all files from a selected dir
    subdir = f"{tmp_dir}/{doc_subdir}"
    for src_dir, dirs, files in os.walk(subdir):
        for file_ in files:
            if include_doc_file_type(file_) and src_dir == subdir and suffix_exc in file_:
                src_file = os.path.join(src_dir, file_)
                dst_file = os.path.join(destination_folder, file_)
                if os.path.exists(dst_file):
                    os.remove(dst_file)
                shutil.copy(src_file, destination_folder)

    # Delete temp clone
    shutil.rmtree(tmp_dir)

    processed_files = []
    failed_files = []

    gen_doc_summary_prompt = gen_doc_summary_prompt.replace("<repo_name>", code_repo)
    
    logger.info(f"Processing doc files in {destination_folder}")
    for file in os.listdir(destination_folder):
        file_path = os.path.join(destination_folder, file)
        filename = file.split(".")[0]
        functionality = filename.replace(suffix_exc, "").capitalize()
        eval_path = f"{eval_folder}/{filename}"
        if not os.path.exists(eval_path):
            os.makedirs(eval_path)
        for attempt in range(3):
            try:
                logger.info(f"\033[92mProcessing doc file: {file}\033[0m")
                gen_doc_summary_prompt_func = gen_doc_summary_prompt.replace("<functionality>", functionality)
                ai_doc = ask_question_to_gen_doc_summary(gen_doc_summary_prompt_func)
                human_doc = open(file_path, 'r').read()
                eval_results = ask_question_to_compare_docs(doc_eval_prompt, human_doc, ai_doc)
                save_answers(ai_doc, "AI-Generated-Doc", eval_path)
                save_answers(human_doc, "Human-Generated-Doc", eval_path)
                save_answers(eval_results, "Evaluation", eval_path)
                shutil.move(file_path, eval_path)
                processed_files.append(file)
                break
            except Exception as e:
                logger.error(f"Error: {e}")
                time.sleep(15)
        else:
            logger.info(f"\033[93mSkipping file: {file_path}\033[0m")
            failed_files.append(file_path)

    logger.info(f"Processed files: {processed_files}")
    logger.info(f"Failed files: {failed_files}")

In [ ]:
code_repo_name = process_code_repo_files(code_repo, file_doc_gen_prompts)

2024-03-28 10:54:36 INFO Cloning repository... https://github.com/bayer-int/dse-apis.git
2024-03-28 10:54:47 INFO Finished cloning repository https://github.com/bayer-int/dse-apis.git
2024-03-28 10:54:48 INFO Processing code files in repositories/code
2024-03-28 10:54:48 INFO Processing code file: repositories/code/Makefile
2024-03-28 10:55:26 INFO Processing code file: repositories/code/go.sum
2024-03-28 10:56:36 INFO Processing code file: repositories/code/go.mod
2024-03-28 10:57:10 INFO Processing code file: repositories/code/buf.gen.yaml
2024-03-28 10:57:30 INFO Processing code file: repositories/code/buf.lock
2024-03-28 10:57:50 INFO Processing code file: repositories/code/config/util.go
2024-03-28 10:58:22 INFO Processing code file: repositories/code/generated/graphql/generated.go
2024-03-28 10:58:52 INFO Processing code file: repositories/code/generated/proto/accounts/v1/accounts_dynamo.pb.go
2024-03-28 10:59:14 INFO Processing code file: repositories/code/generated/proto/accoun

In [34]:
code_repo_name = "dse-apis"
process_doc_repo_files(doc_repo, doc_repo_subdir, suffix, file_doc_summary_gen_prompt, file_doc_eval_prompt, code_repo_name)

2024-03-28 13:43:33 INFO Cloning repository... https://github.com/bayer-int/DSE-DevDocs.git
2024-03-28 13:43:34 INFO Finished cloning repository https://github.com/bayer-int/DSE-DevDocs.git
2024-03-28 13:43:34 INFO Processing doc files in repositories/doc
2024-03-28 13:43:34 INFO Processing doc file: EventsAPI.md
2024-03-28 13:43:46 INFO Processing doc file: modelsAPI.md
2024-03-28 13:43:59 INFO Processing doc file: ApplicationsAPI.md
2024-03-28 13:44:12 INFO Processing doc file: DataScienceProductsAPI.md
2024-03-28 13:44:23 INFO Processing doc file: tenantsAPI.md
2024-03-28 13:44:36 INFO Processing doc file: DeploymentsAPI.md
2024-03-28 13:44:46 INFO Processing doc file: DecisionsAPI.md
2024-03-28 13:44:59 INFO Processing doc file: PromotionsAPI.md
2024-03-28 13:45:09 INFO Processed files: ['EventsAPI.md', 'modelsAPI.md', 'ApplicationsAPI.md', 'DataScienceProductsAPI.md', 'tenantsAPI.md', 'DeploymentsAPI.md', 'DecisionsAPI.md', 'PromotionsAPI.md']
2024-03-28 13:45:09 INFO Failed files